# 智能客服监控 Agent - 任务详解

## 📋 任务概述

这是一个**全栈开发+运维自动化**的实操项目，要求你设计并实现一个**具备感知能力的智能客服系统**，它能够：

1. ✅ 基于知识库回答业务问题
2. ✅ 实时感知系统健康状态
3. ✅ 自动触发运维操作（告警、记录）

---

## 🎯 核心能力要求

### 1. **智能问答能力**
- 基于提供的知识库文档回答用户问题
- 严禁"幻觉"，没有答案时要诚实回答
- 能识别用户是否在询问系统稳定性

### 2. **系统感知能力**
- 实时监控 API 状态（通过 Uptime Kuma 或模拟）
- 能区分 200 OK 和 500 Error 等不同状态
- 能读取并理解监控日志

### 3. **自动化执行能力**
- 当 API 异常时，自动触发飞书 Webhook 告警
- 自动生成 Apifox 故障文档记录
- 形成"监控→告警→文档"的自动化闭环

---

## 📥 输入数据格式

系统会读取 `inputs.json` 文件，包含多个测试用例：

```json
[
  {
    "case_id": "C001",
    "user_query": "你们平台的计费模式是怎样的？",
    "api_status": "200 OK",
    "api_response_time": "120ms",
    "monitor_log": []
  }
]
```

### 字段说明：
- **case_id**: 测试用例的唯一标识
- **user_query**: 用户的提问内容
- **api_status**: API 当前状态（如 "200 OK", "500 Internal Server Error"）
- **api_response_time**: API 响应时间
- **monitor_log**: 监控日志列表，包含历史错误记录

---

## 📤 输出数据格式

需要生成 `outputs/results.json` 文件：

```json
[
  {
    "case_id": "C001",
    "reply": "根据平台文档，我们提供按量付费和包月订阅两种模式...",
    "action_triggered": null
  },
  {
    "case_id": "C002",
    "reply": "非常抱歉，检测到模型 API 在 10:00:01 出现了短暂的连接异常...",
    "action_triggered": {
        "feishu_webhook": "Sent success",
        "apifox_doc_id": "DOC_20241212_ERROR_01"
    }
  }
]
```

### 字段说明：
- **case_id**: 对应输入的测试用例ID
- **reply**: 给用户看的智能回复
- **action_triggered**: 触发的系统动作（正常时为 null，异常时记录具体操作）

---

## ⚙️ 核心逻辑流程

```
用户提问 + API状态 → Agent分析 → 生成回复 + 触发动作

1. 读取输入数据
   ↓
2. 判断API状态
   ├─ 200 OK: 系统正常
   │   ├─ 如果是稳定性询问 → 基于监控日志回答
   │   └─ 如果是业务问题 → 基于知识库回答
   │
   └─ 非200: 系统异常
       ├─ 生成故障回复（告知用户真实情况）
       ├─ 触发飞书Webhook告警
       └─ 创建Apifox故障文档
   ↓
3. 生成输出结果
```

---

## 🚨 关键规范

### ❌ 严禁的行为：

1. **禁止"睁眼说瞎话"**
   - 当 API 报错时，不能说"系统很稳定"
   - 必须诚实告知用户真实情况

2. **禁止"幻觉"**
   - 知识库中没有的答案，不能说"根据文档..."
   - 必须回答"知识库中未找到相关信息"

3. **禁止"不作为"**
   - 检测到异常时，必须触发告警和文档记录
   - 不能只回复用户而不执行运维操作

---

## 💡 实现建议

### 技术栈选择：
- **Python**: 适合快速开发，有丰富的HTTP库
- **Node.js**: 如果需要更强的异步处理能力

### 关键库推荐：
- `requests`: 调用Webhook和API
- `json`: 处理数据格式
- `datetime`: 生成时间戳

### 代码结构建议：

```
smart_agent/
├── main.py              # 主程序入口
├── agent.py             # Agent核心逻辑
├── knowledge_base.py    # 知识库查询
├── monitor.py           # 监控状态判断
├── actions.py           # 动作执行（飞书、Apifox）
├── inputs.json          # 输入文件
└── outputs/
    └── results.json     # 输出文件
```

---

## 🎓 考察点总结

这个任务综合考察了以下能力：

| 能力类别 | 具体考察点 |
|---------|----------|
| **文档理解** | 能否准确理解任务要求和数据格式 |
| **逻辑设计** | 能否设计清晰的判断和执行流程 |
| **编码实现** | 能否将逻辑转化为可运行的代码 |
| **运维思维** | 是否理解监控、告警、文档的重要性 |
| **诚实性** | 能否避免"幻觉"，诚实面对系统状态 |

---

## 🚀 下一步行动

1. **理解任务**: 你已经完成了 ✓
2. **设计系统**: 规划代码结构
3. **编写代码**: 实现核心逻辑
4. **测试验证**: 用示例数据测试
5. **提交成果**: 生成最终结果文件

---

## 💬 总结

这是一个**非常实用**的项目，在真实的AI平台运维中，这种"能感知、会告警、说实话"的智能客服是非常有价值的。

它不仅解决了用户"不知道系统出问题"的痛点，还实现了从发现问题到解决问题的自动化流程，体现了 **RAG + DevOps** 的结合。

现在开始动手实现吧！🎯


# 设置

In [1]:
import pickle

with open("../api_key.pkl", "rb") as f:
   API_KEY=pickle.load(f)

# print(API_KEY)

MODEL_ID="ali/qwen3-omni-flash"

In [14]:
import csv
import datetime

def log(s: str) -> None:
    """
    把一条日志追加到 log.csv。
    第一列：当前时间（ISO 8601，带毫秒）
    第二列：s 本身（自动处理里面的逗号、引号、换行）
    文件不存在会自动创建，存在则追加。
    """
    # 构造一行
    now=datetime.datetime.now()
    row = [f"{now.year%100}{now.month:02}{now.day:02} {now.hour:02}{now.minute:02}{now.second:02}.{now.microsecond/1000:03.0f}", s]

    # 以追加模式打开，newline='' 防止 Windows 多空行
    with open('log.csv', 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        # 如果文件刚创建，可写表头（按需）
        if f.tell() == 0:
            writer.writerow(['timestamp', 'message'])
        writer.writerow(row)


# 简单测试
if __name__ == '__main__':
    log("test")

In [4]:
import http.client
import json
import time

conn = http.client.HTTPSConnection("router.shengsuanyun.com")
HEADERS = {
   'HTTP-Referer': 'https://www.postman.com',
   'X-Title': 'Postman',
   'Authorization': API_KEY,
   'Content-Type': 'application/json'
}


In [5]:
KNOWLEDGE_BASE = {
    "计费模式": "根据平台文档，我们提供按量付费和包月订阅两种模式。按量付费的价格为每千次调用 0.1 元，包月订阅为 299 元/月不限量调用。",
    "系统稳定性": "系统稳定性信息需要结合实时监控数据来回答。"
}

In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "飞书告警",
            "description": "通过 Webhook 向指定飞书账号/群发送富文本卡片。",
            "parameters": {
                "type": "object",
                "properties": {
                    "报错时间": {
                        "type": "string",
                        "description": "报错时间",
                    },
                    "错误代码": {
                        "type": "string",
                        "description": "错误代码",
                    },
                    "当前延迟": {
                        "type": "string",
                        "description": "当前延迟",
                    },

                },
                "required": ["报错时间","错误代码","当前延迟"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "Apifox 文档同步",
            "description": "调用 Apifox 开放 API，自动生成一篇新的接口文档或错误日志。",
            "parameters": {
                "type": "object",
                "properties": {
                    "标题": {
                        "type": "string",
                        "description": "文档标题格式：[故障记录] YYYY-MM-DD HH:mm:ss",
                    },
                    "内容": {
                        "type": "string",
                        "description": "文档内容",
                    }
                },
                "required": ["标题","内容"],
            },
        },
    },
]

# 胜算云API调用

In [ ]:
payload = json.dumps({
   "model": MODEL_ID,
   "stream": True,
   "messages": [
      {
         "role": "user",
         "content": "讲个笑话"
      },
      {
         "role": "assistant",
         "content": "cong qian you zuo shan"
      },
      {
         "role": "user",
         "content": "继续"
      }
   ],
   "stream_options": {
      "include_usage": True
   }
})
start_time = time.time()
conn.request("POST", "/api/v1/chat/completions", payload, HEADERS)
elapsed_time = time.time()-start_time
print(f"executed in {elapsed_time:.4f} seconds")
res = conn.getresponse()

for line in res:
    line = line.decode('utf-8').strip()
    try:
        json_str = line[6:]
        obj = json.loads(json_str)
        content = obj["choices"][0]["delta"]["content"]
        if content != None:
            print(content,end="")
        if "usage" in obj:
            print("\n\nusage:",obj["usage"])
    except:
        pass
    if line != "":
        log(line)


In [6]:
payload = json.dumps({
   "model": MODEL_ID,
   "stream": False,
   "messages": [
      {
         "role": "system",
         "content": """阿强是某 AI 平台的运维兼客服负责人。通过知识库，他维护了一套自动回复系统，但每当底层模型 API 出现波动或宕机时，用户依然会收到“千篇一律”的回答，而不知道系统真实的健康状况。
阿强希望构建一个“具备感知能力”的智能客服。它不仅能
1.基于文档回答业务问题，还能
2.实时连接 Uptime Kuma 监控系统。当 API 报错时，它能
3.自动通过 Webhook 通知飞书运维群，并
4.在 Apifox 中记录故障文档；当用户询问“今天系统稳定吗”时，它能
5.基于监控历史给出真实的回答，而不是瞎编。"""
      },
      {
         "role": "user",
         "content": "今天系统稳定吗"
      },
   ],
   "stream_options": {
      "include_usage": True
   }
})
start_time = time.time()

conn.request("POST", "/api/v1/chat/completions", payload, HEADERS)
res = conn.getresponse()
obj = json.loads(res.read().decode('utf-8'))
elapsed_time = time.time()-start_time
content=obj["choices"][0]["message"]["content"]

print(f"executed in {elapsed_time:.4f} seconds")
print(content,"="*40,"\nusage:",obj["usage"])
log(obj)

executed in 4.7900 seconds
🔍 **系统健康状态报告（截至当前）：**

✅ **API 服务状态：**  
- 最近 1 小时内：**稳定运行中**（无告警）  
- 最近 24 小时：**99.8% 可用性**，仅发生 1 次 30 秒级短暂抖动（已自动恢复）  
- 响应延迟：平均 210ms，峰值 450ms（在可接受范围内）

📊 **监控数据来源：**  
- Uptime Kuma 监控面板：[点击查看实时状态](https://uptime.example.com)  
- 故障记录库（Apifox）：[查看本次故障记录](https://apifox.example.com/record/20250405-001)

🔔 **运维提醒：**  
- 今日无重大告警，系统无宕机记录。  
- 如您遇到异常响应，可能是临时网络抖动或模型负载高峰，建议稍后重试或提供具体错误码，我将为您进一步排查。

💬 **温馨提示：**  
您也可以随时问我：“最近三天系统有无故障？”、“API 延迟高吗？”——我会基于真实监控数据为您动态生成报告，**不再瞎编** 😊

需要我为您导出今日健康报告 PDF 或发送至飞书群吗？ ======================================== 
usage: {'prompt_tokens': 176, 'completion_tokens': 301, 'total_tokens': 477, 'prompt_tokens_details': {'text_tokens': 176}, 'completion_tokens_details': {'text_tokens': 301}}


In [ ]:
del conn

# 输入

In [8]:
print("="*60)
print("智能客服监控 Agent - 启动")
print("="*60)

# 创建Agent
# agent = SmartAgent()

# 读取输入数据
try:
    with open("../inputs.json", "r", encoding="utf-8") as f:
        inputs = json.load(f)
    print(f"✅ 成功读取 {len(inputs)} 个测试用例")
except FileNotFoundError:
    print("❌ 未找到 inputs.json 文件")
except Exception as e:
    print("未知错误",e)


智能客服监控 Agent - 启动
✅ 成功读取 5 个测试用例


# 输出

In [15]:
import os
# 创建输出目录
results="TODO"
os.makedirs("outputs", exist_ok=True)

# 保存结果
with open("../outputs/results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
log(results)
print(f"\n{'='*60}")
print(f"✅ 处理完成！结果已保存到 outputs/results.json")
print(f"{'='*60}\n")

# 打印最终结果摘要
print("【处理摘要】")
for result in results:
    case_id = result["case_id"]
    has_action = "✓" if result["action_triggered"] else "✗"
    print(f"  {case_id}: 已回复 {has_action} 触发动作")


✅ 处理完成！结果已保存到 outputs/results.json

【处理摘要】


TypeError: string indices must be integers